# 梅苗苗状态变化

## 模块配置

In [1]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")  # 忽略UserWarning兼容性警告

yesterday = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
now = datetime.date.today().strftime("%Y-%m-%d")
print(now,yesterday)

Discount_yesterday = float('1.0')
Discount_now       = float('1.0')
df_jp              = pd.read_excel(f'C:/Users/15516/Desktop/数据处理/竞对数据/比对目录——2023.07.24版.xlsx',sheet_name='比价明细')
df_mmm_yesterday   = pd.read_csv(f'C:/Users/15516/Desktop/数据处理/竞对数据/mmm/梅苗苗{yesterday}.csv')
df_mmm_now         = pd.read_csv(f'C:/Users/15516/Desktop/数据处理/竞对数据/mmm/梅苗苗{now}.csv')

2023-07-28 2023-07-27


## 函数配置

In [2]:
def deal_with(df,Discount):

    # 清除型号列中的空格
    df['型号'] = df['型号'].str.strip()

    # 合并品名和型号列，并在型号列为空时只保留品名
    df['品型'] = df['品名']+df['型号'].fillna('')


    # 根据折扣价和价格列判断折扣价格列的值
    df['折扣价格'] = df.apply(lambda row: row['折扣价'] if pd.notnull(row['折扣价']) and row['折扣价'] != 0 else row['价格'] * Discount, axis=1)

    # 重新排列列的顺序
    df = df[['产品编码', '品型', '价格', '折扣价格', '库存量', '限购', '标签']]
    
    return df

def zhuijia(file,key):
    
    # 遍历除了产品编码列之外的所有列
    for col in file.columns:
        if col != '产品编码':
            file.rename(columns={col: col + f'-{key}'}, inplace=True)
            
    return file

## 数据处理

In [3]:
file_yesterday = deal_with(df_mmm_yesterday,Discount_yesterday)
file_now       = deal_with(df_mmm_now,Discount_now)

# 合并并去重产品编码列
df_combined = pd.concat([file_now['产品编码'], file_yesterday['产品编码']]).drop_duplicates().reset_index(drop=True)

# 创建新的数据框
df_merged = pd.DataFrame({'产品编码': df_combined})

# 对file_yesterday进行操作
file_yesterday = zhuijia(file_yesterday, "前")

# 对file_now进行操作
file_now = zhuijia(file_now, "后")

# 进行左连接
df_merged = df_merged.merge(file_yesterday, how='left', on='产品编码').merge(file_now, how='left', on='产品编码')

# 添加上下架状态列
df_merged['上下架状态'] = df_merged.apply(lambda row: '新上架' if pd.isnull(row['品型-前']) else ('下架' if pd.isnull(row['品型-后']) else '上架'), axis=1)

# 填充前后库存的空值为0
df_merged['库存量-前'] = df_merged['库存量-前'].fillna(0)
df_merged['库存量-后'] = df_merged['库存量-后'].fillna(0)

# 添加库存变化列
df_merged['库存变化状态'] = df_merged.apply(lambda row: '上升' if row['库存量-后'] > row['库存量-前'] else ('不变' if row['库存量-后'] == row['库存量-前'] else '下降'), axis=1)

# 计算库存变动量
df_merged['库存变动量'] = df_merged.apply(lambda row: row['库存量-后'] - row['库存量-前'] if row['库存变化状态'] != '不变' else '', axis=1)

# 添加折扣价格变化列
df_merged['折扣价格变动'] = df_merged.apply(lambda row: '涨价' if row['折扣价格-后'] > row['折扣价格-前'] else ('不变' if row['折扣价格-后'] == row['折扣价格-前'] else '降价'), axis=1)

# 计算折扣价格变动量
df_merged['折扣价格变动量'] = df_merged.apply(lambda row: row['折扣价格-后'] - row['折扣价格-前'] if row['折扣价格变动'] != '不变' else '', axis=1)

# 左连接df_jp表
df_merged = df_merged.merge(df_jp, how='left', left_on='产品编码', right_on='梅苗苗产品编码')

df_merged = df_merged[['产品编码','品型-前','品型-后','价格-前','价格-后','折扣价格-前','折扣价格-后','库存量-后','上下架状态','库存变化状态','折扣价格变动','库存变动量','折扣价格变动量','限购-后','标签-后','励齿编码']]

df_merged.to_excel(f'C:/Users/15516/Desktop/梅苗苗变化{now}.xlsx', index=False)

In [4]:
df_merged

,产品编码,品型-前,品型-后,价格-前,价格-后,折扣价格-前,折扣价格-后,库存量-后,上下架状态,库存变化状态,折扣价格变动,库存变动量,折扣价格变动量,限购-后,标签-后,励齿编码
0,0-121960-137011,一品 盐酸罗哌卡因注射液10ml：100mg*5支/盒,一品 盐酸罗哌卡因注射液10ml：100mg*5支/盒,43.0,43.0,43.0,43.0,49.0,上架,下降,不变,-1.0,,NaN,NaN,NaN
1,0-103128-143002,上海朝晖 盐酸利多卡因注射液5ml:0.1g*5支 （近效期）,上海朝晖 盐酸利多卡因注射液5ml:0.1g*5支 （近效期）,4.9,4.9,4.9,2.9,100.0,上架,下降,降价,-400.0,-2.0,100.0,限时特惠 麻药,NaN
2,0-103128-102698,上海朝晖 盐酸利多卡因注射液5ml:0.1g*5支,上海朝晖 盐酸利多卡因注射液5ml:0.1g*5支,4.9,4.9,4.9,4.9,500.0,上架,不变,不变,,,NaN,限时特惠 麻药,NaN
3,0-121378-142130,遂成 韩都 盐酸利多卡因注射液20ml:0.4g*5支/盒,遂成 韩都 盐酸利多卡因注射液20ml:0.4g*5支/盒,10.8,10.8,10.8,10.8,374.0,上架,下降,不变,-3.0,,NaN,NaN,NaN
4,0-108330-141999,上海禾丰 盐酸布比卡因注射液5ml：37.5mg 5支/盒,上海禾丰 盐酸布比卡因注射液5ml：37.5mg 5支/盒,88.8,88.8,88.8,88.8,25.0,上架,不变,不变,,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30802,3-115357-101713,严迪/YANDY 五档电动牙刷粉色,NaN,78.0,NaN,45.0,NaN,0.0,下架,不变,降价,,NaN,NaN,NaN,NaN
30803,1-109287-112837,康田正/COTISEN 吸唾管 弱吸20包/箱,NaN,260.0,NaN,220.0,NaN,0.0,下架,下降,降价,-136.0,NaN,NaN,NaN,NaN
30804,3-113568-121606,精工 口腔印模消毒器JG-33318-YM,NaN,2800.0,NaN,2800.0,NaN,0.0,下架,下降,降价,-500.0,NaN,NaN,NaN,NaN
30805,1-120762-140374,义获嘉伟瓦登特 氟保护剂0.4ml/瓶*40瓶/盒,NaN,471.8,NaN,471.8,NaN,0.0,下架,下降,降价,-1.0,NaN,NaN,NaN,NaN
